### Run in collab
<a href="https://colab.research.google.com/github/racousin/data_science_practice/blob/master/website/public/modules/data-science-practice/module9/exercise/module9_exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install swig==4.2.1
!pip install gymnasium==1.2.0

In [7]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
import random, json

# module9_exercise2 : ML - Arena <a href="https://ml-arena.com/viewcompetition/5" target="_blank"> FrozenLake Competition</a>

### Objective
Get at list an agent running on ML-Arena <a href="https://ml-arena.com/viewcompetition/5" target="_blank"> FrozenLake Competition</a> with mean reward upper than 0.35 (ie 35%)


You should submit an agent file named `agent.py` with a class `Agent` that includes at least the following attributes:

In [3]:
class Agent:
    def __init__(self, env):
        self.env = env

    def choose_action(self, observation, reward=0.0, terminated=False, truncated=False, info=None):
        action = self.env.action_space.sample() # your logic here
        return action

### Description

The game starts with the player at location [0,0] of the frozen lake grid world with the goal located at far extent of the world [7,7].

Holes in the ice are distributed in set locations.

The player makes moves until they reach the goal or fall in a hole.

Each run will consist of 10 attempts to cross the ice. The reward will be the total amount accumulated during those trips. For example, if your agent reaches the goal 3 times out of 10, its reward will be 3.

The environment is based on :

In [4]:
env = gym.make('FrozenLake-v1', map_name="8x8")

In [8]:
def entrainer_agent(nb_episodes=50000, alpha=0.1, gamma=0.99, eps_debut=1.0, eps_fin=0.01, pas_reduction=200000):
    env = gym.make("FrozenLake-v1", map_name="8x8")
    nb_etats = env.observation_space.n
    nb_actions = env.action_space.n
    Q = np.zeros((nb_etats, nb_actions))
    epsilon = eps_debut
    for episode in range(nb_episodes):
        etat, _ = env.reset(seed=episode)
        termine = faux = False

        while not (termine or faux):

            if random.random() < epsilon:
                action = env.action_space.sample()
            else:
                meilleures = np.flatnonzero(Q[etat] == Q[etat].max())
                action = int(random.choice(meilleures))

            nouvel_etat, recompense, termine, faux, _ = env.step(action)


            meilleure_suite = Q[nouvel_etat].max()
            cible = recompense + (0 if (termine or faux) else gamma * meilleure_suite)
            Q[etat, action] += alpha * (cible - Q[etat, action])
            etat = nouvel_etat
        if episode < pas_reduction:
            epsilon = eps_debut - (eps_debut - eps_fin) * (episode / pas_reduction)
        else:
            epsilon = eps_fin

    env.close()
    return Q


def tester_agent(Q, nb_runs=200, essais_par_run=10):

    env = gym.make("FrozenLake-v1", map_name="8x8")
    total_runs = []
    for r in range(nb_runs):
        total = 0
        for _ in range(essais_par_run):
            etat, _ = env.reset()
            termine = faux = False
            while not (termine or faux):
                meilleures = np.flatnonzero(Q[etat] == Q[etat].max())
                action = int(random.choice(meilleures))
                etat, recompense, termine, faux, _ = env.step(action)
                total += recompense
        total_runs.append(total)
    env.close()

    moyenne_run = np.mean(total_runs)
    taux_succes = moyenne_run / essais_par_run
    return moyenne_run, taux_succes


#je teste et on va faire la moyenne
Q = entrainer_agent()
moyenne, succes = tester_agent(Q)
print("reward moyenne :", round(moyenne, 2))
print("succès moyen  :", round(succes, 3), "≈", round(100*succes,1), "%")

reward moyenne : 6.15
succès moyen  : 0.615 ≈ 61.5 %


### Before submit
Test that your agent has the right attributes

In [12]:
env = gym.make('FrozenLake-v1', map_name="8x8")
agent = Agent(env)

observation, _ = env.reset()
reward, terminated, truncated, info = None, False, False, None
rewards = []
while not (terminated or truncated):
    action = agent.choose_action(observation, reward=reward, terminated=terminated, truncated=truncated, info=info)
    observation, reward, terminated, truncated, info = env.step(action)
    rewards.append(reward)
print(f'Cumulative Reward: {sum(rewards)}')

Cumulative Reward: 1.0
